In [1]:
import numpy as np
from tensorflow.keras.layers import Dense,LSTM,Embedding,Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
def getdata(path):
    ip_text=[]
    op_text=[]
    for line in open(path):
        ip_line = "<start> "+line
        op_line=line+" <end>"
        ip_text.append(ip_line)
        op_text.append(op_line)
    return ip_text,op_text
        
    

In [3]:
ip_text,op_text = getdata("poetry.txt")

In [4]:
op_text[0]


'Two roads diverged in a yellow wood,\n <end>'

In [5]:
ip_text[0]

'<start> Two roads diverged in a yellow wood,\n'

In [6]:
all_text = ip_text+op_text

In [7]:
tokenizer = Tokenizer(num_words=5000,filters='')
tokenizer.fit_on_texts(all_text)

In [8]:
word_to_index=(tokenizer.word_index) #dictionary 

In [9]:
ip_seq = tokenizer.texts_to_sequences(ip_text)
op_seq = tokenizer.texts_to_sequences(op_text)

In [10]:
max_len=0
for item in ip_seq:
    max_len=max(len(item),max_len)
max_len   

13

In [11]:
ip_padded_seq = pad_sequences(ip_seq,maxlen=13)
op_padded_seq = pad_sequences(op_seq,maxlen=13)


In [12]:
ip_padded_seq[0]

array([  0,   0,   0,   0,   0,   1, 101, 506, 507,  10,   8, 508, 890])

In [13]:
op_padded_seq[0]

array([  0,   0,   0,   0,   0, 101, 506, 507,  10,   8, 508, 890,   2])

In [14]:
#Word Vectors Loading
word2vec = {}
with open('C:/Users/Asus/Desktop/ML PROJECTS/Glove Vectors/glove.6B.100d.txt',"r",encoding="utf-8") as f:
    for lines in f:
        value = lines.split()
        word=value[0]
        vector=np.asarray(value[1:],dtype="float32")
        word2vec[word]=vector


In [15]:
max_vocab_size=20000
numwords=min(max_vocab_size,len(word_to_index)+1) # as keras tokenizes from 1
emb_matrix=np.zeros((numwords,100)) #100 dim vectors
for word,index in word_to_index.items():
    if index<max_vocab_size:
        vector = word2vec.get(word) #don't use [''] to prevent eroor,.get returns null if word not found.
        if vector is not None:
            emb_matrix[index]=vector

In [16]:
n_seq=len(ip_seq)

In [17]:
#For training
one_hot_vectors=np.zeros((n_seq,max_len,numwords))#n_Seq 2D arrays with rows being 13 and cols being numwords
for index,vector in enumerate(op_padded_seq):
    for wordind,word in enumerate(vector):
        if(word>0):
            one_hot_vectors[index][wordind][word]=1 #go to right sentence,go to appropirate wordindex,set 1 to the correct output.
            

In [18]:
#embedding layer
embedding = Embedding(numwords,100,weights=[emb_matrix],trainable=False)

In [19]:
inputs = Input(shape=(13))
initial_h=Input(shape=(50))
initial_c=Input(shape=(50))
x=embedding(inputs) #13*100 each ip
lstm = LSTM(50,return_sequences=True,return_state=True) #13*50 op now
x,_,_ = lstm(x,initial_state=[initial_h,initial_c]) 
dense = Dense(numwords,activation='softmax')
op=dense(x)
model = Model([inputs,initial_h,initial_c],op)

In [20]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 13)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 13, 100)      332400      input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 50)]         0                                            
______________________________________________________________________________________________

In [21]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
checkpoint = ModelCheckpoint('poetrybest.h5',save_best_only=True)

In [23]:
z=np.zeros((n_seq,50))
m=model.fit([ip_padded_seq,z,z],one_hot_vectors,batch_size=64,epochs=500,validation_split=0.2,callbacks=[checkpoint])

Train on 1264 samples, validate on 317 samples
Epoch 1/500
1264/1264 [==============================] - 1s 835us/sample - loss: 0.0016 - accuracy: 0.9998 - val_loss: 0.0017 - val_accuracy: 0.9998
Epoch 2/500
1264/1264 [==============================] - 1s 884us/sample - loss: 0.0015 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 3/500
1264/1264 [==============================] - 1s 895us/sample - loss: 0.0015 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 4/500
1264/1264 [==============================] - 1s 799us/sample - loss: 0.0014 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 5/500
1264/1264 [==============================] - 1s 853us/sample - loss: 0.0014 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 6/500
1264/1264 [==============================] - 1s 819us/sample - loss: 0.0014 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 7/500
1264/1264 [==============================] - 1s

1264/1264 [==============================] - 1s 677us/sample - loss: 0.0013 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 56/500
1264/1264 [==============================] - 1s 671us/sample - loss: 0.0013 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 57/500
1264/1264 [==============================] - 1s 680us/sample - loss: 0.0013 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 58/500
1264/1264 [==============================] - 3s 2ms/sample - loss: 0.0013 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 59/500
1264/1264 [==============================] - 1s 747us/sample - loss: 0.0013 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 60/500
1264/1264 [==============================] - 1s 712us/sample - loss: 0.0013 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 61/500
1264/1264 [==============================] - 1s 725us/sample - loss: 0.0013 - accuracy: 0.9998 - val_l

Epoch 110/500
1264/1264 [==============================] - 1s 716us/sample - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 111/500
1264/1264 [==============================] - 1s 673us/sample - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 112/500
1264/1264 [==============================] - 1s 674us/sample - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 113/500
1264/1264 [==============================] - 1s 716us/sample - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 114/500
1264/1264 [==============================] - 1s 673us/sample - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 115/500
1264/1264 [==============================] - 1s 682us/sample - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 116/500
1264/1264 [==============================] - 1s 675us/sample - loss: 0.0012 - ac

1264/1264 [==============================] - 1s 672us/sample - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 165/500
1264/1264 [==============================] - 1s 670us/sample - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 166/500
1264/1264 [==============================] - 1s 672us/sample - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 167/500
1264/1264 [==============================] - 1s 670us/sample - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 168/500
1264/1264 [==============================] - 1s 671us/sample - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 169/500
1264/1264 [==============================] - 1s 674us/sample - loss: 0.0012 - accuracy: 0.9998 - val_loss: 0.0017 - val_accuracy: 0.9998
Epoch 170/500
1264/1264 [==============================] - 1s 672us/sample - loss: 0.0012 - accuracy: 0.9998

1264/1264 [==============================] - 1s 673us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 219/500
1264/1264 [==============================] - 1s 675us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 220/500
1264/1264 [==============================] - 1s 677us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 221/500
1264/1264 [==============================] - 1s 670us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 222/500
1264/1264 [==============================] - 1s 685us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 223/500
1264/1264 [==============================] - 1s 674us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 224/500
1264/1264 [==============================] - 1s 678us/sample - loss: 0.0011 - accuracy: 0.9998

1264/1264 [==============================] - 1s 672us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 273/500
1264/1264 [==============================] - 1s 674us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 274/500
1264/1264 [==============================] - 1s 675us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 275/500
1264/1264 [==============================] - 1s 671us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 276/500
1264/1264 [==============================] - 1s 675us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 277/500
1264/1264 [==============================] - 1s 680us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0015 - val_accuracy: 0.9998
Epoch 278/500
1264/1264 [==============================] - 1s 683us/sample - loss: 0.0011 - accuracy: 0.9998

1264/1264 [==============================] - 1s 671us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 327/500
1264/1264 [==============================] - 1s 677us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 328/500
1264/1264 [==============================] - 1s 673us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 329/500
1264/1264 [==============================] - 1s 666us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 330/500
1264/1264 [==============================] - 1s 680us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 331/500
1264/1264 [==============================] - 1s 679us/sample - loss: 0.0011 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 332/500
1264/1264 [==============================] - 1s 670us/sample - loss: 0.0011 - accuracy: 0.9998

1264/1264 [==============================] - 1s 677us/sample - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 381/500
1264/1264 [==============================] - 1s 674us/sample - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 382/500
1264/1264 [==============================] - 1s 671us/sample - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 383/500
1264/1264 [==============================] - 1s 674us/sample - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 384/500
1264/1264 [==============================] - 1s 676us/sample - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 385/500
1264/1264 [==============================] - 1s 673us/sample - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 386/500
1264/1264 [==============================] - 1s 721us/sample - loss: 0.0010 - accuracy: 0.9998

1264/1264 [==============================] - 1s 690us/sample - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 435/500
1264/1264 [==============================] - 1s 696us/sample - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 436/500
1264/1264 [==============================] - 1s 684us/sample - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 437/500
1264/1264 [==============================] - 1s 679us/sample - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 438/500
1264/1264 [==============================] - 1s 689us/sample - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 439/500
1264/1264 [==============================] - 1s 676us/sample - loss: 0.0010 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 440/500
1264/1264 [==============================] - 1s 679us/sample - loss: 0.0010 - accuracy: 0.9998

1264/1264 [==============================] - 1s 681us/sample - loss: 9.7539e-04 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 488/500
1264/1264 [==============================] - 1s 688us/sample - loss: 9.7528e-04 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 489/500
1264/1264 [==============================] - 1s 679us/sample - loss: 9.7395e-04 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 490/500
1264/1264 [==============================] - 1s 675us/sample - loss: 9.7344e-04 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 491/500
1264/1264 [==============================] - 1s 683us/sample - loss: 9.7301e-04 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 492/500
1264/1264 [==============================] - 1s 679us/sample - loss: 9.7224e-04 - accuracy: 0.9998 - val_loss: 0.0016 - val_accuracy: 0.9998
Epoch 493/500
1264/1264 [==============================] - 1s 686us/sample - loss: 9

In [24]:
model.save_weights('500epoch.h5')

In [25]:
input2=Input(shape=(1))
x = embedding(input2)
x,h,c = lstm(x,initial_state=[initial_h,initial_c])
op2 = dense(x)
sample_model = Model([input2,initial_h,initial_c],[op2,h,c])

idtoword={Id:word for word,Id in word_to_index.items()}



In [28]:
sample_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             332400      input_4[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 50)]         0                                            
____________________________________________________________________________________________

In [31]:
ip_padded_seq.shape
max_len


<tf.Tensor 'lstm_1/Identity:0' shape=(None, 1, 50) dtype=float32>

In [75]:
def sample_line():
    np_ip = np.array([[word_to_index['<start>']]])
    h=np.zeros((1,50))
    c=np.zeros((1,50))
    endtok = word_to_index['<end>']
    op_sent=[]
    for _ in range(max_len):
        o,h,c = sample_model.predict([np_ip,h,c])
        probs = o[0][0]
        probs[0]=0
        probs/=probs.sum()
        idx = np.random.choice(len(probs),p=probs) #element with higher p has more chance to get selected
        if idx==endtok:
            break
        op_sent.append(idtoword.get(idx))
        np_ip[0][0]=idx
    #return probs,o,np_ip
    print(' '.join(op_sent))
        
    
    

In [80]:
for i in range(4):
    sample_line()

panther shake eyes.)
 the life pipe's best struck it i didn't recognize,

names attic,
 one found your would think in this throve

governor rested out out of flowers out used mad. and hold consideration? 

course
 taken do but i can told tempt i'll 'toffile, it peas.



In [93]:
for i in range(4):
    sample_line()

afraid we is are up from the dead 
 the wizard

beside mountain
 come a is well at a four they back to greatness
seemed, our 'the but they heard held in a passing 

grafton proclaimed means that all there back in a show not cost 

